In [3]:
"""
    源自清华大学 github 网站上的案例（https://github.com/THUDM/ChatGLM-6B）

    调试量化 int4级别

    先保证本地安装调试完CUDA，然后创建conda 环境 进行环境配置 pip install -r requirements.txt -i https://mirror.sjtu.edu.cn/pypi/web/simple

"""


from transformers import AutoTokenizer, AutoModel
model_path = "D:\CodeLibrary\ChatGLM\chatglm26b"

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
# model = AutoModel.from_pretrained(model_path, trust_remote_code=True, device='cuda')
# 按需修改，目前只支持 4/8 bit 量化
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).quantize(4).half().cuda()
model = model.eval()

response, history = model.chat(tokenizer, "你好", history=[])
print(response + '\n')

response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
print(response + '\n')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

E:\software\anaconda3\envs\chatGLM_env\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。

睡眠对身体健康和心理健康都很重要，如果你晚上睡不着，可以尝试以下一些方法来帮助自己入睡:

1. 改善睡眠环境:确保睡眠环境安静、舒适、黑暗、凉爽。如果睡眠环境不合适，会感到疲劳和不安，从而更难入睡。

2. 放松身心:在睡觉前一小时，放松身心是很重要的。你可以做些轻松的伸展运动、冥想或进行伸展运动，或做瑜伽。

3. 规律作息:尽量在同一时间入睡和起床，即使在周末也应该保持相同的作息时间，这有助于调节身体的生物钟。

4. 避免刺激:睡觉前一小时，尽量避免看电视、玩游戏或进行刺激性的活动，因为这可能会让你兴奋和不安。

5. 避免咖啡因和酒精:咖啡因和酒精都可能影响睡眠质量，因此在睡觉前一小时，尽量避免摄入这些物质。

6. 远离压力:在睡觉前一小时，尽量避免进行紧张的活动，如激烈的运动或紧张的工作。

如果这些方法不能解决你的问题，可以尝试寻求医生的帮助，找到更好的解决方案。



In [ ]:
response, history = model.chat(tokenizer, "还有其他的吗", history=history)
print(response + '\n')

In [ ]:
import os

# 获取当前工作目录
current_working_directory = os.getcwd()

print("当前工作目录:", current_working_directory)


In [ ]:
import sys

print(sys.prefix)

In [ ]:
# 查看有哪些类
from langchain import embeddings
import inspect
classes = [m[0] for m in inspect.getmembers(embeddings, inspect.isclass)]

print(classes)

In [ ]:
# 查看包里面具体的类 代码
print(inspect.getsource(embeddings.HuggingFaceHubEmbeddings))

# 根据langchain 和 Faiss构建知识向量库       

其中使用 `pip install faiss-gpu` 安装 GPU版本的 Faiss向量数据库



首先引入相关的包

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
import faiss
from langchain.vectorstores import FAISS    
from langchain.embeddings import HuggingFaceEmbeddings
import pickle

In [ ]:
import torch
print(torch.__version__)

print(torch.version.cuda)
print(torch.backends.cudnn.version())

# 取消用 Faiss，window不支持，或者说要求特别麻烦，并且没讲讲清楚固定的依赖，导致无法安装



主要体现在cuda版本，还有支不支持gpu的问题；要在linux上比较容易，故放弃（浪费时间配环境）   

# 换成milvus～    

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
# import milvus
# from langchain.vectorstores import milvus
from langchain.embeddings import HuggingFaceEmbeddings
import pickle

# 加载数据，格式为md文件  

In [7]:
from pathlib import Path
ps = list(Path('D:/Notes/NLP review').glob("**/*.md"))

data = []
sources = []
for p in ps:
    with open(p, encoding='utf8') as f:
        data.append(f.read())
    sources.append(p)
print("data example")
print(data[0])
# print("-"*50)
# print(sources[0])

data example
`Beam search` 是一种启发式搜索算法，常用于自然语言处理中的解码阶段，特别是在机器翻译、语音识别和文本生成等任务中。它的主要目的是在解码过程中保持一个候选列表（即“beam”），这个列表中的每个元素都是一个部分构建的序列（如句子的一部分），并且根据某种评分机制（如概率）对这些候选进行排序和筛选。

`Beam search` 的**核心思想**是在每个时间步（或每个词汇的生成步骤）不仅仅考虑单一的最优候选，而是维护一个大小为 B 的候选集（B 通常被称为 beam width 或 beam size），这样可以在解码过程中探索多条路径。在每一步，算法都会根据当前的候选集扩展出新的候选集，并从中选择评分最高的 B 个候选，作为下一步的候选集。这个过程一直持续到生成序列的结束符被添加到某个候选中，或者达到序列的最大长度限制。


下面通过一个简单的例子来说明 beam search 的过程：
假设我们有一个语言模型，它在每个时间步都会为下一个词汇生成一个概率分布。我们要使用这个模型来生成一个句子，并且设置 beam size 为 3。
1. 第一步，模型生成了三个词汇的概率分布，我们选择概率最高的三个词汇作为当前步的候选：["我"，"你"，"他"]。
2. 第二步，对于每个候选，模型生成接下来一个词汇的概率分布。我们为每个候选词汇组合计算一个联合概率，并选择概率最高的三个组合作为新的候选集。例如，可能的新候选集是：["我 爱"，"你 是"，"他 的"]。
3. 重复这个过程，直到某个候选序列以句子结束符（如句号）结束，或者达到序列的最大长度。


`Beam search` 允许算法在解码过程中考虑多个可能的候选，从而**在一定程度上平衡了贪心算法（每次选择最优候选）和穷举搜索（考虑所有可能的候选）之间的权衡**。通过调整 beam size，我们可以控制算法在解码过程中探索的广度和深度。较大的 beam size 可以提高找到最优解的可能性，但同时也会增加计算成本。
需要注意的是，Beam search 并不保证能找到最优解，因为它可能会在解码过程中丢弃最终的最优路径。此外，Beam search 也不保证能得到多样性很强的输出，因为它倾向于选择概率较高的候选，这可能会导致生成的句子比较单一。为了解决这个问题，有时会采用一些额外的

# 由于 LLM有context限制，故需要spliter，`chunk_size`先随意设置   

- `metadatas` 的作用：**每个分割后的文本块仍然需要保留与原始文本的关联信息**  
- `splits` 是将 text 分块

In [8]:
import uuid
text_splitter = CharacterTextSplitter(chunk_size=1500, separator='\n')
docs = []
metadatas = []
ids = []
for i,d in enumerate(data):
    splits = text_splitter.split_text(d)
    doc_ids = [str(uuid.uuid4()) for _ in splits]  # 生成唯一的 ID
    ids.extend(doc_ids)
    docs.extend(splits)
    metadatas.extend([{"source": str(sources[i])}] * len(splits))
print(docs[0], 'docs[0] lens:', len(docs[0]))
# print("-"*100)
# print(metadatas[0])
# print("-"*100)
# print(ids[0])
# print(type(metadatas))

`Beam search` 是一种启发式搜索算法，常用于自然语言处理中的解码阶段，特别是在机器翻译、语音识别和文本生成等任务中。它的主要目的是在解码过程中保持一个候选列表（即“beam”），这个列表中的每个元素都是一个部分构建的序列（如句子的一部分），并且根据某种评分机制（如概率）对这些候选进行排序和筛选。
`Beam search` 的**核心思想**是在每个时间步（或每个词汇的生成步骤）不仅仅考虑单一的最优候选，而是维护一个大小为 B 的候选集（B 通常被称为 beam width 或 beam size），这样可以在解码过程中探索多条路径。在每一步，算法都会根据当前的候选集扩展出新的候选集，并从中选择评分最高的 B 个候选，作为下一步的候选集。这个过程一直持续到生成序列的结束符被添加到某个候选中，或者达到序列的最大长度限制。
下面通过一个简单的例子来说明 beam search 的过程：
假设我们有一个语言模型，它在每个时间步都会为下一个词汇生成一个概率分布。我们要使用这个模型来生成一个句子，并且设置 beam size 为 3。
1. 第一步，模型生成了三个词汇的概率分布，我们选择概率最高的三个词汇作为当前步的候选：["我"，"你"，"他"]。
2. 第二步，对于每个候选，模型生成接下来一个词汇的概率分布。我们为每个候选词汇组合计算一个联合概率，并选择概率最高的三个组合作为新的候选集。例如，可能的新候选集是：["我 爱"，"你 是"，"他 的"]。
3. 重复这个过程，直到某个候选序列以句子结束符（如句号）结束，或者达到序列的最大长度。
`Beam search` 允许算法在解码过程中考虑多个可能的候选，从而**在一定程度上平衡了贪心算法（每次选择最优候选）和穷举搜索（考虑所有可能的候选）之间的权衡**。通过调整 beam size，我们可以控制算法在解码过程中探索的广度和深度。较大的 beam size 可以提高找到最优解的可能性，但同时也会增加计算成本。
需要注意的是，Beam search 并不保证能找到最优解，因为它可能会在解码过程中丢弃最终的最优路径。此外，Beam search 也不保证能得到多样性很强的输出，因为它倾向于选择概率较高的候选，这可能会导致生成的句子比较单一。为了解决这个问题，有时会采用一些额外的技术，如随机化或长度惩罚，来增加输出

# 使用 faiss 把文本转换为 vector, 选择huggingface开源的模型 text2vec；通过faiss构建存储到磁盘的知识向量库
代码如下：（**因为无法安装faiss导致无法使用**）       
```python           
store = FAISS.from_texts(docs, HuggingFaceEmbeddings(model_name="shibing624/text2vec-base-multilingual"), metadatas=metadatas)
faiss.write_index(store.index, "docs.index")               
store.index = None
with open("faiss_store.pkl", 'wb') as f:
    pickle.dump(store, f)
```

# 发现 milvus不好用，操作太复杂了，使用 chromeadb 平替
```python
import chromadb
from chromadb.utils import embedding_functions
import pickle
from transformers import AutoTokenizer, AutoModel
import torch
import logging

# 忽略警告(如果报错或者那里有有问题，导致查不出原因就删掉试试)   
logging.basicConfig(level=logging.WARNING)

# 初始化 ChromaDB 客户端
client = chromadb.Client()

# 创建一个集合用于存储文档及其嵌入
try:
    client.delete_collection(name="my_collection")
except Exception as e:
    print(f"Error deleting collection: {e}")

collection = client.create_collection(name="my_collection")

# 定义嵌入函数
model_name = r"D:\CodeLibrary\ChatGLM\text2vecmul"


tokenizer = AutoTokenizer.from_pretrained(model_name)
embeddings = AutoModel.from_pretrained(model_name)

# def embed_texts(texts):
#     inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
#     with torch.no_grad():
#         outputs = model(**inputs)
#     embeddings = outputs.last_hidden_state.mean(dim=1).numpy()

#     return embeddings.tolist()

# 获取嵌入向量 

# docs = ["document1", "document2", "document3"]      
# metadatas = [{"meta": "data1"}, {"meta": "data2"}, {"meta": "data3"}]
# embeddings = embed_texts(docs)

# 向集合中添加数据
collection.add(ids=ids, documents=docs, metadatas=metadatas, embeddings=embeddings)

"""
    (因为无法通过pickle序列化 client中的某些链接, 故用dict方式进行数据序列化，但是没用，因为后续其实需要整个db的内容序列化后的加载)
"""
# 提取集合中的数据
collection_data = {                   
    "ids": docs,
    "documents": docs,
    "metadatas": metadatas,
    "embeddings": embeddings,
}

# 将集合数据保存到文件
with open("chroma_collection_data.pkl", 'wb') as f:
    pickle.dump(collection_data, f)
# 关闭客户端（chromadb 没有 close这种操作）   
# client.close()        

print("done!")
```

# 上面的代码又因为是直接用 chromadb，在序列化索引这一块又有问题，不知道怎么解决   

# 故，根据 langchain文档，改用 langchain中 集成的 chroma
- 前提仍然是要安装 `chromadb`
  ```python
  pip install chromadb
  ```
- **不再使用** `client = chromadb.Client()` 初始化客户端，然后再创建 `collection`
- **改成了** 通过导包 `from langchain.vectorstores import Chroma`, 然后通过 `db = Chroma.from_documents（doc, embeddings）` 来获取

# 更新后的代码如下:    

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import torch
from langchain.docstore.document import Document

# embedding_model 和 tokenizer获取方式不变
embed_model_name = r"D:\CodeLibrary\ChatGLM\text2vecmul"
tokenizer = AutoTokenizer.from_pretrained(embed_model_name)
embed_model = AutoModel.from_pretrained(embed_model_name)

def embed_texts(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = embed_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()

    return embeddings.tolist()

# embeddings = embed_texts(docs)
embeddings = HuggingFaceEmbeddings(model_name=embed_model_name)

# 将文本和元数据封装成Document对象  
documents = [Document(page_content=text, metadata=meta) for text, meta in zip(docs, metadatas)]
print(type(metadatas))
store = Chroma.from_documents(documents=documents, embedding=embeddings)
store.persist("chroma_store")  # 持久化存储

# 利用 `inspect` 查看源码    

In [2]:
import inspect
import chromadb
from langchain.chains import RetrievalQA

# 获取源码
source_code = inspect.getsource(RetrievalQA)

print(source_code)


class RetrievalQA(BaseRetrievalQA):
    """Chain for question-answering against an index.

    Example:
        .. code-block:: python

            from langchain.llms import OpenAI
            from langchain.chains import RetrievalQA
            from langchain.faiss import FAISS
            from langchain.vectorstores.base import VectorStoreRetriever
            retriever = VectorStoreRetriever(vectorstore=FAISS(...))
            retrievalQA = RetrievalQA.from_llm(llm=OpenAI(), retriever=retriever)

    """

    retriever: BaseRetriever = Field(exclude=True)

    def _get_docs(
        self,
        question: str,
        *,
        run_manager: CallbackManagerForChainRun,
    ) -> List[Document]:
        """Get docs."""
        return self.retriever.get_relevant_documents(
            question, callbacks=run_manager.get_child()
        )

    async def _aget_docs(
        self,
        question: str,
        *,
        run_manager: AsyncCallbackManagerForChainRun,
    ) -> List[Docum

# 接下来是 访问知识库与用户交互                  

这里使用 langchain API `RetrievalQA` 来访问知识库        

1. 从磁盘加载向量数据库   
```python   
index = faiss.read_index('./docs.index') # 之前生成的数据库index



with open("./faiss_store.pkl", 'rb') as f:   
    store = pickle.load(f)   

store.index = index   
```

2. 设计一个prompt 模板   
```python   
template = """基于以下信息来回答用户问题。                            
已知信息：   
{context}   
问题：
{question}"""    
```

3. 使用 `RetrievalQA` 来 基于对知识库的访问结果 与 LLM 进行交互    
```python                                           
prompt = PromplateTemplate(template=template, input_variables=["context", "question"])




chain_type_kwargs = {"prompt":prompt}    
qa = RetrievalQA.from_chain_type(llm=chatglm, retriever=store.as_retriever(), chain_type="stuff",
                                chain_type_kwargs=chain_type_kwargs, return_source_documents=True)



```

4. 构建一个查询输入框，从LLM返回结果中，提取问题答案和知识库文档引用来源：   
```python   
query = input("\n请输入问题：")                         

res = qa(query)   
answer, doc = res['result'], res['source_documents']




print('\n\n> 问题：')     
print(query)    
print("\n> 回答：")         
print(answer)     
print("\n> 来源：")                        
for document in docs:  
    print("\n> " + str(document.metadata['source']) + ":")


```

# chroma 版本的 知识库访问 与 用户交互
1. chroma 索引加载
2. prompt template 定义  
3. 用户交互定义

In [ ]:
store = Chroma.load("chroma_store", embeddings)

template = """基于以下信息来回答用户问题。                          
已知信息： 
{context} 
问题：
{question}"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt":prompt}
qa = RetrievalQA.from_chain_type(llm=chatglm, retriever=store.as_retriever(), chain_type="stuff",
                                chain_type_kwargs=chain_type_kwargs, return_source_documents=True)

# 运行

In [ ]:
query = input("\n请输入问题：")                       

res = qa(query) 
answer, doc = res['result'], res['source_documents']

print('\n\n> 问题：')   
print(query)  
print("\n> 回答：")       
print(answer)   
print("\n> 来源：")                      
for document in docs:
    print("\n> " + str(document.metadata['source']) + ":")
